<a href="https://colab.research.google.com/github/2SEHI/K-Digital_Lecture/blob/main/0818_pyTorch(RNN%2C%20NPL)/pyTorch(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN

## CNN과 RNN의 쓰임
- CNN : 이미지 처리. 일부분을 가지고 전체를 판단하며 비교할 때 사용
- RNN : 시계열 예측, 자연어 처리에 사용되며 과거의 데이터로 미래를 예측하는데에 쓰입니다. 예를 들어가격그래프를 보고 앞으로 어떻게 될것인가를 예측합니다


## 1.RNN(Recurrent Neural Network)이란?
- 순환 신경망이라는 의미
- "TRIANGEL"이라는 단어와 "INTEGRAL"이라는 단어가 존재하는 경우 단어의 구성 알파벳은 같지만 순서가 다르므로 서로 다른 단어입니다.
- 단어, 알파벳, 데이터 등의 **순서**에 따라 숨겨진 패턴을 찾아서 상관 관계나 인과 관계를 찾고자 시작한 것이 RNN입니다. 
- 지금까지 배운 일반적인 인공 신경망은 입력이 주어지면 은닉층에서 어떤 연산을 수행해서 다음 은닉층으로 넘겨서 연산을 수행한 후 출력을 만들어줬습니다.
- 그러나 RNN은 이전 상태가 다음 은닉층으로 전달되어 은닉층에서 계산을 할 때 이전에 수행한 연산과 현재 입력값과의 조합이 같이 계산됩니다.
- t=0 의 경우는 어떤 초기값과 계산이 시작을 해서 은닉층의 값들을 계산하고 결과값이 만들어지면 다음 은닉층으로 전송을 하고 현재 계산된 값을 보존하고 있다가 다음 계산을 수행하는 t=1시점에서 현재의 입력과 이전에 계산한 값을 가지고 다시 결과를 생성해서 다음 은닉층에 전달합니다. 이 과정으로 time step을 지정한 만큼 반복을 하게됩니다.

## 2.BPTT(Back Propagation Throught Time, 역전파)
- RNN에서는 역전파를 할 때 이전과 다르게 t=0에서 t=3까지의 연산을 수행했다고 하면 현재 시점까지 전체에 대해서 역전파를 해야 합니다. 
- pytorch단어를 가지고 입력을 하나씩 받아서 다음 글자를 예측하는 시스템을 만든다고 합시다. DNN이나 CNN구조를 이용한다면 p라는 입력이 온 경우 다음 글자 y를 예측해야 하는데 이 때 만일 y를 정확하게 예측했다면 py를 가지고 다음 t를 예측합니다. t가 아닌 글자를 예측했다면 잘못된 예측을 했으므로 손실을 계산해서 역전파를 해야하는데 이 때 y에게만 영향을 줍니다.
- RNN모델에서는 y-> 그 이전에게도 역전파를 합니다.
- I am a poy 역전파를 하지 않을 경우 a만 보는데 RNN의 경우 I am a 이전의 모든 요소에 대해 영향이 없는지 확인합니다.

2. 일반적으로 활성화함수로 하이퍼볼릭 탄젠트 함수를 이용합니다. 역전파를 하면 시간이 오래 걸립니다.
3. RNN은 데이터가 많을 수록 정확한 모델이 만들어집니다.

## 3. 기본적인 RNN의 문제점
- 입력이 커지면 (문장의 길이가 길어지거나 예측하기 위해서 훈련하는 데이터의 길이가 길어지면) 기울기 소실 문제가 발생합니다. 
- 기울기 소실 (gradient vanishing) : 역전파나 순방향 훈련시 tanh(하이퍼볼릭 탄젠트 함수)를 이용하는데 이 함수는 미분을 하게 되면 0에서 1사이의 값이 나오게 되고 이를 여러번 곱하면 기울기가 0이 될 가능성이 높아집니다.


## 4. 기울기 소실 문제의 해결책
- 하이퍼볼릭 탄젠트 함수를 사용하지 않고 활성화함수로 변경하는 방법
- LSTM이나 GRU모델을 이용하는 방법 
    - 단순히 미분을 하는 것이 아니라 그 이전 은닉층의 cell 상태를 기억해서 사용하는 방식입니다.
    - LSTM은 cell 상태라는 것을 별도로 갖지만 GRU는 이를 단순화해서 RNN과 비슷하게 만들고 LSTM의 성능과 유사한 성능을 나타내도록 한 것입니다.


## 5. RNN모델을 이용한 문장예측


### 공통처리

#### 1) 필요한 라이브러리 import

In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim

import numpy as np

#### 2) 하이퍼 파라미터 설정
- 학습률 : 경사하강법에서의 보폭
- 훈련횟수 : 훈련횟수는 많은게 좋지만 너무 많으면 시간이 오래걸립니다

In [ ]:
n_hidden = 35 # 은닉층
lr = 0.01 # 학습률
epochs = 1000 # 훈련횟수

#### 3) 샘플데이터와 분류 토큰 생성

In [ ]:
# 샘플 데이터와 분류를 위한 토큰을 생성
string = 'hello pytorch. how long can a rnn cell remember? show me your limit!'
# 앞의 문자들은 문장에 포함된 글자들이고 뒤의 0과 1은 시작과 종료를 위한 문자입니다.
chars = 'abcedfghijklmnopqrstuvwxyz ?!.:;01'

# 글자의 개수 저장
char_list = [i for i in chars]
n_letters = len(char_list)
n_letters

34

#### 4) One hot Encoding을 위한 함수생성
- 대문자는 에러가 발생하므로 lower를 이용하여 소문자로 만들어서 매개변수에 전달해야합니다

In [ ]:
def string_to_onehot(string):
    # 시작과 끝 토큰 생성
    start = np.zeros(shape=n_letters, dtype=int)
    end = np.zeros(shape=n_letters, dtype=int)

    # 맨 마지막에 end를 그 앞에 start를 표현하기 위해서 그 자리에만 1을 추가
    start[-2] = 1
    end[-1] = 1

    #입력된 문자를 순회
    for i in string:
        # 글자의 인덱스 찾기 - 없는 글자가 있으면 예외 발생
        idx = char_list.index(i)
        # 각 글자를 표현하기 위한 배열 생성
        zero = np.zeros(shape=n_letters, dtype=int)
        zero[idx] = 1 
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

# 현재는 대문자 넣으면 에러 남
# print(string_to_onehot('HI'))
# 대문자는 lower를 해주면 에러가 안남
print(string_to_onehot('HI'.lower()))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


#### 5) 원핫 인코딩된 문자열을 복원해주는 함수

In [ ]:
# 원핫 인코딩된 문자열을 복원해주는 함수
def onehot_to_word(onehot_1):
    # 원핫 인코딩된 숫자들을 Tensor로 변환하고 
    # 가장 큰 인덱스에 해당하는 글자를 가지고 리스트를 생성합니다
    onehot = torch.Tensor.numpy(onehot_1)
    # print(onehot)
    # print(onehot.argmax())
    return char_list[onehot.argmax()]


### RNN 구현

#### 1) RNN 모델 설계

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        # 인스턴스 변수(속성, 프로퍼티)를 생성해서 값을 대입
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # 입력 크기에 자신의 hidden_size를 더해줘야 합니다.
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

        # RNN의 경우 활성화 함수는 하이퍼뵐릭 tahn함수를 사용합니다.
        self.act_fn = nn.Tanh()

    def forward(self, input, hidden):
        # 현재 입력과 이전 출력을 붙이기
        combined = torch.cat((input, hidden), 1)
        # 역전파를 위한 hidden 상태의 업데이트
        hidden = self.act_fn(self.i2h(combined))
        # 출력만들기
        output = self.i2o(combined)

        return output, hidden

    # 아직 입력이 없을때(t=0)의 hidden state를 초기화해줍니다. 
    def init_hidden(self):
        # print(self.hidden_size)
        return torch.zeros(1, self.hidden_size)

#### 2) 모델 생성

In [ ]:
# 모델 생성
rnn = RNN(n_letters, n_hidden, n_letters)

#### 3) 손실함수와 optimizer 설정

In [ ]:
# 손실함수 생성
loss_func = nn.MSELoss()
# 최적화 함수는 Adam을 사용
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

#### 4) 모델 훈련

In [ ]:
# 문자열을 원 핫 벡터로 만들고 텐서로 변경
one_hot = torch.from_numpy(string_to_onehot('hi')).type_as(torch.FloatTensor())
# print(one_hot.size()[0]-1)
# 훈련
for i in range(epochs):
    # optimizer(최적화 함수) 초기화
    optimizer.zero_grad()
    # 학습에 앞서 hidden state를 초기화해줍니다.
    hidden = rnn.init_hidden()
    
    # 문자열 전체에 대한 손실을 구하기 위해 total_loss라는 변수를 만들어줍니다. 
    total_loss = 0
    for j in range(one_hot.size()[0]-1):
        # 입력은 앞에 글자
        # pyotrch 에서 p y t o r c
        input_ = one_hot[j:j+1,:]
        # print(input_)
        # 목표값은 뒤에 글자
        # pytorch 에서 y t o r c h
        target = one_hot[j+1]
        output, hidden = rnn.forward(input_, hidden)
        
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss
    # print('전체 손실 : ', total_loss)
    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(total_loss)


tensor(0.1052, grad_fn=<AddBackward0>)
tensor(0.0233, grad_fn=<AddBackward0>)
tensor(0.0127, grad_fn=<AddBackward0>)
tensor(0.0063, grad_fn=<AddBackward0>)
tensor(0.0027, grad_fn=<AddBackward0>)
tensor(0.0009, grad_fn=<AddBackward0>)
tensor(0.0003, grad_fn=<AddBackward0>)
tensor(5.1936e-05, grad_fn=<AddBackward0>)
tensor(6.6821e-06, grad_fn=<AddBackward0>)
tensor(2.0990e-06, grad_fn=<AddBackward0>)
tensor(2.6904e-06, grad_fn=<AddBackward0>)
tensor(1.6642e-06, grad_fn=<AddBackward0>)
tensor(6.2977e-07, grad_fn=<AddBackward0>)
tensor(1.3137e-07, grad_fn=<AddBackward0>)
tensor(1.1937e-08, grad_fn=<AddBackward0>)
tensor(5.8756e-09, grad_fn=<AddBackward0>)
tensor(6.7681e-09, grad_fn=<AddBackward0>)
tensor(3.2215e-09, grad_fn=<AddBackward0>)
tensor(7.4169e-10, grad_fn=<AddBackward0>)
tensor(7.4048e-11, grad_fn=<AddBackward0>)
tensor(4.6833e-11, grad_fn=<AddBackward0>)
tensor(4.5324e-11, grad_fn=<AddBackward0>)
tensor(1.6630e-11, grad_fn=<AddBackward0>)
tensor(2.2091e-12, grad_fn=<AddBackward

#### 5) 모델 테스트

In [ ]:
# hidden state 는 처음 한번만 초기화해줍니다.
start = torch.zeros(1,n_letters)
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    # 처음 입력으로 start token을 전달해줍니다.
    input_ = start
    # output string에 문자들을 계속 붙여줍니다.
    output_string = ""

    # 원래는 end token이 나올때 까지 반복하는게 맞으나 끝나지 않아서 string의 길이로 정했습니다.
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        # 결과값을 문자로 바꿔서 output_string에 붙여줍니다.
        output_string += onehot_to_word(output.data)
        # 또한 이번의 결과값이 다음의 입력값이 됩니다.
        input_ = output

print(output_string)